In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "<cliendid>",
"fs.azure.account.oauth2.client.secret": "<secretvalue>",
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<tanentid>/oauth2/token"}

## create key vault instead of giving keys in script

In [ ]:
# %python
# Unmount the existing directory
dbutils.fs.unmount("/mnt/tempfolder")

# Mount the directory again
dbutils.fs.mount(
    source="abfss://containername@storageaccountname.dfs.core.windows.net", ## container@storageaccount
    mount_point="/mnt/tempfolder",
    extra_configs=configs
)

/mnt/tempfolder has been unmounted.


True

In [ ]:
%fs
ls "/mnt/tempfolder"

path,name,size,modificationTime
dbfs:/mnt/tempfolder/pretransformed_data/,pretransformed_data/,0,1737318346000
dbfs:/mnt/tempfolder/transformed_data/,transformed_data/,0,1737318361000


In [ ]:
spark

In [ ]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tempfolder/pretransformed_data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tempfolder/pretransformed_data/coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tempfolder/pretransformed_data/entriesgender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tempfolder/pretransformed_data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tempfolder/pretransformed_data/teams.csv")

In [ ]:
entriesgender = entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))
     

In [ ]:
 # Find the top countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("TeamCountry","Gold").show()
     

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  40|
|People's Republic...|  40|
|               Japan|  20|
|           Australia|  18|
|              France|  16|
|         Netherlands|  15|
|       Great Britain|  14|
|   Republic of Korea|  13|
|               Italy|  12|
|             Germany|  12|
|         New Zealand|  10|
|              Canada|   9|
|          Uzbekistan|   8|
|             Hungary|   6|
|               Spain|   5|
|              Sweden|   4|
|               Kenya|   4|
|              Norway|   4|
|             Ireland|   4|
|              Brazil|   3|
+--------------------+----+
only showing top 20 rows



In [ ]:

# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|3x3 Basketball, B...|     1|   0|    1|                1.0|                0.0|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    94|  96|  190|0.49473684210526314| 0.5052631578947369|
|   Artistic Swimming|   106|   0|  106|                1.0|                0.0|
|           Athletics|   982|1038| 2020| 0.4861386138613861| 0.5138613861386139|
|           Badminton|    87|  88|  175|0.49714285714285716| 0.5028571428571429|
|          Basketball|   143| 144|  287|0.49825783972125437| 0.5017421602787456|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   124

In [ ]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tempfolder/transformed_data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tempfolder/transformed_data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tempfolder/transformed_data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tempfolder/transformed_data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tempfolder/transformed_data/teams")